# CSV Analyzer Agent
Loads and summarizes a CSV using a tool.

In [ ]:
import pandas as pd
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

# Define a tool to load and analyze CSV files
@tool
def load_csv(path: str) -> str:
    """Loads a CSV file and returns statistical summary.
    
    Args:
        path: The file path to the CSV file
    
    Returns:
        A string containing statistical summary of the CSV data
    """
    try:
        df = pd.read_csv(path)
        summary = f"CSV loaded successfully!\n\n"
        summary += f"Shape: {df.shape[0]} rows, {df.shape[1]} columns\n\n"
        summary += f"Columns: {', '.join(df.columns.tolist())}\n\n"
        summary += f"Statistical Summary:\n{df.describe().to_string()}"
        return summary
    except Exception as e:
        return f"Error loading CSV: {str(e)}"

# Initialize the LLM
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# Get the ReAct prompt template from LangChain hub
prompt = hub.pull("hwchase17/react")

# Create the agent
tools = [load_csv]
agent = create_react_agent(llm, tools, prompt)

# Create the executor
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# Test the agent (Note: you'll need to create a sample.csv file or change the path)
result = executor.invoke({'input': 'Load and analyze the CSV at sample.csv'})
print(result)